In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from requests import get
from csv import writer

In [2]:
url = 'https://www.txdirectory.com/online/dist/'
response = requests.get(url).text

In [3]:
# Creating BeautifulSoup object
soup = BeautifulSoup(response, 'html.parser')

In [4]:
print('Classes of each table:')
for table in soup.find_all('table'):
    print(table.get('class'))

Classes of each table:
['table', 'table-condensed', 'table-striped']


In [5]:
# Creating list with all tables
tables = soup.find_all('table')

#  Looking for the table with the classes 'wikitable' and 'sortable'
table = soup.find(class_='table table-condensed table-striped')

In [ ]:
table

In [6]:
#find all rows
rows = table.find_all('tr')

In [7]:
#interate through rows and columns
for row in rows[1:]:
    cells = row.find_all(['td','th'])
    
    cells_text = [cell.get_text(strip = True) for cell in cells]
    print(cells_text)

['1', 'The Honorable Craig M. Mixson(R)', 'Jasper, Newton, Sabine, San Augustine', '(409) 384-3792 /(409) 384-9722']
['1-A', 'The Honorable Delinda Gibbs-Walker(R)', 'Jasper, Newton, Tyler', '(409) 384-9570 /(409) 384-9722']
['2', 'The Honorable Chris Day(R)', 'Cherokee', '(903) 683-2236 /(903) 683-2238']
['3', 'The Honorable Mark A. Calhoon(R)', 'Anderson, Henderson, Houston', '(903) 723-7415 /(903) 723-7494']
['4', 'The Honorable J. Clay Gossett(R)', 'Rusk', '(903) 657-0358 /(903) 655-1250']
['5', 'The Honorable Bill Miller(R)', 'Bowie, Cass', '(903) 628-6783 /(903) 628-2217']
['6', 'The Honorable Ronald Wesley "Wes" Tidwell(R)', 'Lamar, Red River', '(903) 737-2432 /(903) 737-2483']
['7', 'The Honorable Kerry L. Russell(R)', 'Smith', '(903) 590-1643 /(903) 590-1641']
['8', 'The Honorable Eddie Northcutt(R)', 'Delta, Franklin, Hopkins, Rains', '(903) 438-4022 /(903) 438-4092']
['9', 'The Honorable Phil Grant(R)', 'Montgomery', '(936) 539-7866 /(936) 788-8381']
['10', 'The Honorable Ke

In [8]:
#write to csv
with open('district_numbers.csv', 'w', encoding='utf8', newline ='') as f:
    thewriter = writer(f)
    header = ['district', 'judge', 'county', 'phone']
    thewriter.writerow(header)
    
    for row in rows[1:]:
        cells = row.find_all(['td','th'])
    
        cells_text = [cell.get_text(strip = True) for cell in cells]
        thewriter.writerow(cells_text)

In [9]:
df = pd.read_csv('district_numbers.csv')
df

,district,judge,county,phone
0,1,The Honorable Craig M. Mixson(R),"Jasper, Newton, Sabine, San Augustine",(409) 384-3792 /(409) 384-9722
1,1-A,The Honorable Delinda Gibbs-Walker(R),"Jasper, Newton, Tyler",(409) 384-9570 /(409) 384-9722
2,2,The Honorable Chris Day(R),Cherokee,(903) 683-2236 /(903) 683-2238
3,3,The Honorable Mark A. Calhoon(R),"Anderson, Henderson, Houston",(903) 723-7415 /(903) 723-7494
4,4,The Honorable J. Clay Gossett(R),Rusk,(903) 657-0358 /(903) 655-1250
...,...,...,...,...
469,482,The Honorable Maritza Antu(R),Harris,(832) 927-4040
470,485,The Honorable Steven Jumes(R),Tarrant,NaN
471,505,The Honorable Kali Morgan(D),Fort Bend,(281) 238-3244 /(281) 238-3255
472,506,The Honorable Gary W. Chaney(R),"Grimes, Waller",(979) 921-0921 /(979) 399-5033


In [ ]:
df_melt = df.assign(new_county=df.county.str.split(","))
df_melt

In [ ]:
df_melt.new_county.apply(pd.Series)

In [ ]:
df_melt.new_county.apply(pd.Series) \
    .merge(df_melt, right_index = True, left_index = True)

In [ ]:
df_melt.new_county.apply(pd.Series)\
.merge(df_melt, right_index = True, left_index = True)\
.drop(['county', 'new_county'], axis = 1)\
.melt(id_vars = ['district', 'judge', 'phone'], value_name = 'county')

In [11]:
df_melt = df.assign(new_county=df.county.str.split(","))
df_melt = df_melt.new_county.apply(pd.Series) \
    .merge(df_melt, right_index=True, left_index=True) \
    .drop(['county', 'new_county'], axis=1) \
    .melt(id_vars=['district', 'judge', 'phone'], value_name = 'county') \
    .drop("variable", axis=1) \
    .dropna()
  
print(df_melt)

     district                                    judge  \
0           1         The Honorable Craig M. Mixson(R)   
1         1-A    The Honorable Delinda Gibbs-Walker(R)   
2           2               The Honorable Chris Day(R)   
3           3         The Honorable Mark A. Calhoon(R)   
4           4         The Honorable J. Clay Gossett(R)   
...       ...                                      ...   
2287      394         The Honorable Roy B. Ferguson(D)   
2344      452          The Honorable Robert Hofmann(R)   
2394       24            The Honorable Jack W. Marr(R)   
2506      135        The Honorable Stephen Williams(R)   
2638      267  The Honorable Robert E. "Bobby" Bell(R)   

                               phone     county  
0     (409) 384-3792 /(409) 384-9722     Jasper  
1     (409) 384-9570 /(409) 384-9722     Jasper  
2     (903) 683-2236 /(903) 683-2238   Cherokee  
3     (903) 723-7415 /(903) 723-7494   Anderson  
4     (903) 657-0358 /(903) 655-1250       Rusk  
...

In [12]:
df_melt

,district,judge,phone,county
0,1,The Honorable Craig M. Mixson(R),(409) 384-3792 /(409) 384-9722,Jasper
1,1-A,The Honorable Delinda Gibbs-Walker(R),(409) 384-9570 /(409) 384-9722,Jasper
2,2,The Honorable Chris Day(R),(903) 683-2236 /(903) 683-2238,Cherokee
3,3,The Honorable Mark A. Calhoon(R),(903) 723-7415 /(903) 723-7494,Anderson
4,4,The Honorable J. Clay Gossett(R),(903) 657-0358 /(903) 655-1250,Rusk
...,...,...,...,...
2287,394,The Honorable Roy B. Ferguson(D),(432) 837-5831 /(432) 837-1241,Presidio
2344,452,The Honorable Robert Hofmann(R),(325) 347-0755 /(325) 347-0756,Menard
2394,24,The Honorable Jack W. Marr(R),(361) 575-3172 /(361) 572-5682,Victoria
2506,135,The Honorable Stephen Williams(R),(361) 575-2412 /(361) 572-5682,Victoria


In [17]:
df_melt.to_csv('split_counties.csv')

Resources
https://www.txdirectory.com/online/dist/
https://www.geeksforgeeks.org/convert-list-like-column-elements-to-separate-rows-in-pandas/
